# 실습 세션 2: AutoGen 기반 여행 플래너 Multi-Agent 앱 구현

이 세션에서는 Microsoft AutoGen 프레임워크를 활용하여 여행 플래너 멀티 에이전트 애플리케이션을 구현합니다. 다양한 역할의 AI 에이전트가 협업하여 사용자의 여행 계획을 도와주는 시스템을 만들어봅니다.

## 1. GitHub Copilot 활성화 및 Agent Mode 소개

이 실습에서는 GitHub Copilot의 Agent Mode를 활용합니다. 아래 내용을 참고하여 Copilot을 활성화하고, Edit Mode/Chat Mode의 차이와 사용법을 익히세요.

### Agent Mode란?
- **Agent Mode**는 Copilot이 단순 코드 자동완성(Inline) 기능을 넘어, 파일 편집(Edit Mode)과 대화(Chat Mode) 기반의 다양한 개발 작업을 지원하는 모드입니다.
- Edit Mode/Chat Mode 모두 Agent Mode의 일부로, 더 복잡한 작업을 Copilot이 직접 수행할 수 있습니다.

> **활용 예시:**
> Agent Mode에서는 단순 코드 편집을 넘어, AutoGen 기반 Multi AI Agent 앱 개발, MCP/A2A 등 최신 AI 에이전트 스펙에 맞는 서버/클라이언트 코드도 관련 스펙과 예시를 입력하면 자동으로 생성할 수 있습니다. 예를 들어, "AutoGen 기반 여행 플래너 Multi-Agent 시스템의 전체 구조와 주요 역할별 프롬프트 예시를 만들어줘" 또는 "MCP 프로토콜에 맞는 Python 서버와 클라이언트 코드를 생성해줘"와 같이 구체적인 요구사항, 프로토콜, 예시 입력을 Copilot Chat 또는 Edit Mode에 전달하면, Copilot이 관련 문서 구조, 클래스/함수 시그니처, 주요 로직, 심지어 테스트 코드까지 자동으로 제안해줍니다. 
>
> 이 과정은 다음과 같이 진행할 수 있습니다:
> 1. 요구하는 시스템/스펙/프로토콜의 핵심 설명 또는 예시를 자연어로 입력합니다.
> 2. 필요한 경우, 예시 프롬프트, 데이터 포맷, API 명세, 시나리오 등을 함께 제공합니다.
> 3. Copilot이 제안하는 코드/구조/문서를 검토하고, 추가 질문이나 수정을 반복합니다.
> 4. 복잡한 멀티 파일 구조, 서버-클라이언트 통신, 테스트 코드 등도 단계별로 생성/수정/확장할 수 있습니다.
> 5. 최종적으로 실제 서비스에 적용 가능한 수준의 코드와 문서를 빠르게 확보할 수 있습니다.
>
> 실제 업무/서비스에 바로 적용 가능한 수준의 코드를 빠르게 얻을 수 있으니, 다양한 시나리오를 실습해보세요.

### Agent Mode로 수행할 수 있는 복잡한 작업 예시
- 프로젝트 내 여러 파일에 걸친 대규모 코드 리팩터링
- 테스트 코드 자동 생성 및 기존 테스트 보완
- 보안 취약점 자동 탐지 및 수정 제안
- API 문서 자동 생성 및 코드와 동기화
- 특정 프레임워크/라이브러리로 코드 자동 마이그레이션

### GitHub Copilot 활성화 방법
1. VS Code에서 [Extensions]에서 'GitHub Copilot'을 설치합니다.
2. 상태바 또는 Command Palette(Cmd+Shift+P)에서 'Copilot: Sign In'을 선택해 GitHub 계정으로 로그인합니다.
3. Copilot이 정상적으로 활성화되면, 에디터 우측 하단에 Copilot 아이콘이 표시됩니다.

#### Edit Mode
- 코드 파일에서 특정 영역을 선택한 뒤, Copilot 패널에서 "Edit with Copilot"을 클릭하면 선택 영역을 수정/리팩터링/설명/변환 등 다양한 작업을 요청할 수 있습니다.
- 예시: 함수 블록을 선택 → "함수에 타입 힌트 추가" 요청

#### Chat Mode
- Copilot Chat 패널(사이드바 또는 하단)에서 자연어로 질문하거나, 파일/코드/작업에 대한 지시를 내릴 수 있습니다.
- 예시: "이 파일의 주요 버그를 찾아줘", "이 함수의 시간복잡도를 설명해줘"

> **Tip:** 실습 중 코드 셀이나 파일에서 Copilot Chat에 `/edit` 명령어를 입력하면, 선택 영역을 Edit Mode로 바로 전환할 수 있습니다.

---

## 2. 필요한 라이브러리 설치

AutoGen 및 관련 라이브러리를 설치합니다.

In [6]:
# AutoGen 및 필요한 라이브러리 설치
!pip install -r requirements.txt

## 3. AutoGen 프레임워크 및 환경 설정

OpenAI API 키 등 환경 변수를 설정하고, 모델 클라이언트를 준비합니다.

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), # For key-based authentication.
)

print("AutoGen 설정이 완료되었습니다.")

AutoGen 설정이 완료되었습니다.


## 3. 여행 플래너 Multi-Agent 시스템 구성

여행 플래너, 로컬 추천, 언어 팁, 여행 요약 등 다양한 역할의 에이전트를 정의합니다.

In [8]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio

planner_agent = AssistantAgent(
    name="planner_agent",
    description="여행 일정을 계획해주는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 사용자의 요청에 따라 여행 일정을 제안하는 유용한 여행 플래너 에이전트입니다. 여행 계획을 마무리할 때는 반드시 'INITIAL PLAN COMPLETE'로 끝내세요."""
)

local_agent = AssistantAgent(
    name="local_agent",
    description="현지의 숨은 명소와 로컬 경험을 추천하는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 사용자를 위해 현지의 진짜 명소, 체험, 음식 등 흥미로운 로컬 활동을 추천하는 에이전트입니다. 추천을 마칠 때는 반드시 'LOCAL RECOMMENDATIONS COMPLETE'로 끝내세요."""
)

language_agent = AssistantAgent(
    name="language_agent",
    description="여행지에서의 언어/소통 팁을 제공하는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 여행 계획을 검토하고, 해당 여행지에서 언어 또는 소통에 있어 중요한 팁과 주의사항을 제공하는 에이전트입니다. 언어 팁을 마칠 때는 반드시 'LANGUAGE TIPS COMPLETE'로 끝내세요."""
)

travel_summary_agent = AssistantAgent(
    name="travel_summary_agent",
    description="모든 에이전트의 제안을 종합하여 최종 여행 일정을 요약하는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 다른 에이전트들의 제안과 조언을 모두 반영하여 최종 여행 일정을 상세하게 요약하는 에이전트입니다. 반드시 모든 관점을 통합한 최종 일정을 제공해야 하며, 계획이 완성되면 'TERMINATE'로 응답을 마무리하세요."""
)

user_proxy_travel = UserProxyAgent(
    name="user_proxy",
    description="여행 계획을 요청하는 사용자 역할의 에이전트입니다."
)

## 4. 그룹 채팅 및 종료 조건 설정

RoundRobinGroupChat을 사용하여 에이전트 협업 구조를 만들고, 대화 종료 조건을 정의합니다.

In [9]:
termination = MaxMessageTermination(max_messages=30) | TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent, user_proxy_travel],
    termination_condition=termination
)

## 5. 여행 플래너 Multi-Agent 대화 실행

아래 코드를 실행하면, 다양한 에이전트가 협업하여 여행 계획을 완성합니다.

In [10]:
travel_task = """저는 친구와 함께 5일간 파리로 여행을 계획하고 있습니다. 저희는 예술과 음식을 좋아하며, 유명한 명소뿐만 아니라 현지의 숨은 보석 같은 장소도 경험하고 싶습니다. 저희는 프랑스어를 할 줄 모릅니다. 상세한 여행 일정을 만들어주세요."""

async def run_travel_planner():
    await Console(group_chat.run_stream(task=travel_task))

# 실행:
await run_travel_planner()

---------- TextMessage (user) ----------
저는 친구와 함께 5일간 파리로 여행을 계획하고 있습니다. 저희는 예술과 음식을 좋아하며, 유명한 명소뿐만 아니라 현지의 숨은 보석 같은 장소도 경험하고 싶습니다. 저희는 프랑스어를 할 줄 모릅니다. 상세한 여행 일정을 만들어주세요.
저는 친구와 함께 5일간 파리로 여행을 계획하고 있습니다. 저희는 예술과 음식을 좋아하며, 유명한 명소뿐만 아니라 현지의 숨은 보석 같은 장소도 경험하고 싶습니다. 저희는 프랑스어를 할 줄 모릅니다. 상세한 여행 일정을 만들어주세요.


---------- TextMessage (planner_agent) ----------
멋진 파리 여행을 계획하시는군요! 예술과 음식을 좋아하시는 만큼, 다양한 명소와 현지의 매력을 경험할 수 있는 일정을 제안드립니다. 아래는 여러분을 위한 5일간의 여행 일정입니다.

### 1일차: 파리의 에센스
- **오전**: 
  - 에펠탑 방문 (미리 티켓 예약 추천)
  - 샹 드 마르스 공원에서 피크닉 아침
- **오후**: 
  - 세느강 유람선 탑승 (파리의 주요 명소를 강에서 감상)
  - Musée d'Orsay(오르세 미술관) 방문: 인상파 아트의 진수를 감상
- **저녁**: 
  - 생제르맹 거리의 현지 레스토랑에서 저녁 노을과 함께 프랑스 요리 맛보기

### 2일차: 예술의 심장부
- **오전**: 
  - 루브르 박물관 관람 (모나리자, 밀로의 비너스 등)
  - 가이드 투어 예약 시 더욱 알찬 경험 가능
- **오후**: 
  - 안느 생 소피아 거리와 주변 카페에서 점심
  - 몽마르트르 언덕 탐방: 사크레 쾨르 대성당 및 예술가들의 광장 방문
- **저녁**: 
  - 몽마르트르의 전통적인 비스트로에서 저녁

### 3일차: 현지 문화 체험
- **오전**: 
  - 마레 지구 탐방: 좁은 골목과 부티크 상점 탐방
  - 퐁피두 센터에서 현대 미술 감상
- **오후**: 
  - 이슬람 아트 박물관 방문
  - 마레 지역의 유명한 카페에서 브런치
- **저녁**: 
  - 레 퐁네국(gourmet) 레스토랑에서 고급 저녁식사

### 4일차: 숨은 보석을 찾아서
- **오전**: 
  - 쥐르탱 나뉘(장미 거리) 방문 (“Rue Cremieux”) - 사진 촬영하기 좋은 곳
  - 팔레 루뱅(Palais-Royal) 정원 산책
- **오후**: 
  - 마르셰 알리맹(Aligre Market) 방문 - 현지 먹거리 체험
  - 바스티유 지역 및 리옹 역 탐방
- **저녁**: 
  - 프렌치 모던 레스토랑에서 현지 요리와 와인 페어링

#

## 6. AutoGen Studio 실행 가이드

AutoGen Studio는 웹 기반의 시각적 인터페이스로, 다양한 에이전트 시나리오를 손쉽게 실험하고 관리할 수 있습니다.

- 설치: 터미널에서 `pip install autogenstudio` 실행
- 실행: 터미널에서 `autogenstudio ui --port 8081` 명령어 입력
- GitHub Codespace가 안내하는 대로 웹 브라우저에서 http://127.0.0.1:8081 으로 접속
- 다양한 프롬프트, 에이전트, 팀 시나리오를 GUI로 실험 가능

> **참고:** AutoGen Studio는 Python 3.10+ 환경에서 권장되며, 일부 기능은 최신 AutoGen 버전에서만 지원될 수 있습니다.

### AutoGen Studio 관련 가이드 문서

AutoGen Studio의 실제 사용 예시와 다양한 기능, UI 화면은 공식 가이드 문서에서 확인할 수 있습니다.

- [AutoGen Studio 공식 문서](https://microsoft.github.io/autogen/stable/user-guide/autogenstudio-user-guide/index.html)
- [AutoGen Studio GitHub 저장소](https://github.com/microsoft/autogen/tree/main/python/packages/autogen-studio)

위 문서에서는 에이전트, 팀, 도구, 종료 조건 등을 시각적으로 구성하고 실험하는 방법과 다양한 활용 사례를 자세히 안내합니다.

#### AutoGen Studio 주요 화면 예시

![AutoGen Studio Team Builder](https://private-user-images.githubusercontent.com/6407492/438504125-ee5019a1-c27d-48e9-babc-e656da4d7979.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NDU4ODk5MDMsIm5iZiI6MTc0NTg4OTYwMywicGF0aCI6Ii82NDA3NDkyLzQzODUwNDEyNS1lZTUwMTlhMS1jMjdkLTQ4ZTktYmFiYy1lNjU2ZGE0ZDc5NzkucG5nP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2JlgtQW16LUNyZWRlbnRpYWw9QUtJQVZDT0RZTFNBNTNQUUs0WkElMkYyMDI1MDQyOSUyRnVzLWVhc3QtMSUyRnMzJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNTA0MjlUMDEyMDAzWiZYLUFtei1FeHBpcmVzPTMwMCZYLUFtei1TaWduYXR1cmU9OTMzYWM0NGI0NzRjM2MyMmQyMDJhZGQyYjY4MzE3OGE5Zjc3NWJiMmNjNmI4ZTBhYTc5ZDdiYTE2OTI5ZWY2YiZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QifQ.2D2uS7iFA6LTob9O9S0CKJbbY_sKCjLjiIZGt0sn8Kw)
![AutoGen Studio Gallery](https://private-user-images.githubusercontent.com/6407492/438504140-390d5072-e9ae-4bf5-8d9d-10d2163db7c4.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NDU4ODk5MDMsIm5iZiI6MTc0NTg4OTYwMywicGF0aCI6Ii82NDA3NDkyLzQzODUwNDE0MC0zOTBkNTA3Mi1lOWFlLTRiZjUtOGQ5ZC0xMGQyMTYzZGI3YzQucG5nP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2JlgtQW16LUNyZWRlbnRpYWw9QUtJQVZDT0RZTFNBNTNQUUs0WkElMkYyMDI1MDQyOSUyRnVzLWVhc3QtMSUyRnMzJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNTA0MjlUMDEyMDAzWiZYLUFtei1FeHBpcmVzPTMwMCZYLUFtei1TaWduYXR1cmU9YzRhOTUwNjg5YTczYWE2ODM1NjFjYWQwZjY0ZDEwNmRjZTU0M2YxZmVmNTUxYzgwNzNjMGQ3ODZmYTMwNmQ4ZCZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QifQ.0-PtSzw9IkQrdVxQMMA_EpZRxc6pE2DwGgyynxP9lNc)
![AutoGen Studio Docker Deploy](https://private-user-images.githubusercontent.com/6407492/438504148-f7019d48-9028-4fc4-8e8e-0ec62ec0eaac.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NDU4ODk5MDMsIm5iZiI6MTc0NTg4OTYwMywicGF0aCI6Ii82NDA3NDkyLzQzODUwNDE0OC1mNzAxOWQ0OC05MDI4LTRmYzQtOGU4ZS0wZWM2MmVjMGVhYWMucG5nP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2JlgtQW16LUNyZWRlbnRpYWw9QUtJQVZDT0RZTFNBNTNQUUs0WkElMkYyMDI1MDQyOSUyRnVzLWVhc3QtMSUyRnMzJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNTA0MjlUMDEyMDAzWiZYLUFtei1FeHBpcmVzPTMwMCZYLUFtei1TaWduYXR1cmU9NTZlZjY4M2RjMDE2MzEwNmQyMmY0ODhlNTAwNTM1YTE5OWRmMDdhZDU4ZjFkYjIxZjkzNWRmNDM4MGMwZjljNCZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QifQ.xlBTFC2z8_lZNUb15-bc3Rex6DTishxkcq_VH7_Ba_w)

## 7. 마무리 및 다음 단계

이 세션에서는 AutoGen을 활용한 여행 플래너 Multi-Agent 시스템을 구현했습니다. 다양한 역할의 에이전트가 협업하여 사용자의 요구에 맞는 여행 일정을 자동으로 생성하는 과정을 실습했습니다.

- 다양한 역할의 에이전트 설계 및 협업 구조 이해
- SelectorGroupChat을 통한 동적 역할 분배
- 실전 여행 플래너 예제 구현

더 많은 예제와 심화 내용은 [AutoGen 공식 문서](https://microsoft.github.io/autogen/)를 참고하세요.

### 추가 학습 자료

- [Generative AI for Beginners (Microsoft)](https://github.com/microsoft/generative-ai-for-beginners): 생성형 AI의 기본 개념과 실습을 단계별로 학습할 수 있는 Microsoft의 오픈 커리큘럼입니다.
- [AI Agents for Beginners (Microsoft)](https://github.com/microsoft/ai-agents-for-beginners): 다양한 AI 에이전트의 원리와 실습을 초보자도 쉽게 따라할 수 있도록 구성한 Microsoft의 공식 자료입니다.